In [1]:
import pandas as pd
import os

os.chdir(r"C:\Users\Gilseung\Google 드라이브\작업\온라인 강의 머신러닝 성능 향상을 위한 데이터 탐색과 전처리 심화\데이터\실습 데이터 - #3 ~ #5")

In [2]:
df = pd.read_csv("abalone.csv")

In [3]:
# 특징과 라벨 분리
X = df.drop(['Age'], axis = 1)
Y = df['Age']

In [4]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
Train_X.corr() # 특징 간 상관 행렬 출력 => 얼핏봐도 특징 간 선형 관계가 존재

,Length,Diameter,Height,Wholeweight,Shuckedweight,Visceraweight,Shellweight
Length,1.000000,0.986601,0.805464,0.925799,0.896402,0.903029,0.899919
Diameter,0.986601,1.000000,0.810857,0.926071,0.892236,0.900225,0.906323
Height,0.805464,0.810857,1.000000,0.799203,0.754102,0.778551,0.797427
Wholeweight,0.925799,0.926071,0.799203,1.000000,0.968445,0.965756,0.955264
Shuckedweight,0.896402,0.892236,0.754102,0.968445,1.000000,0.930622,0.881972
Visceraweight,0.903029,0.900225,0.778551,0.965756,0.930622,1.000000,0.907741
Shellweight,0.899919,0.906323,0.797427,0.955264,0.881972,0.907741,1.000000


#### VIF 기준 특징 선택

In [6]:
# VIF 계산
from sklearn.linear_model import LinearRegression as LR
VIF_dict = dict()
for col in Train_X.columns:
    model = LR().fit(Train_X.drop([col], axis = 1), Train_X[col])
    r2 = model.score(Train_X.drop([col], axis = 1), Train_X[col]) # LinearRegression의 score가 r2 점수임
    VIF = 1 / (1 - r2)
    VIF_dict[col] = VIF

In [7]:
VIF_dict # Height를 제외하곤 VIF가 모두 높으므로, 이러한 상황에서는 사실 PCA를 사용하는 것이 바람직

{'Diameter': 40.76004936612344,
 'Height': 3.144821205934488,
 'Length': 39.877531276281,
 'Shellweight': 20.25373588264337,
 'Shuckedweight': 26.41132170336298,
 'Visceraweight': 16.726810776990856,
 'Wholeweight': 101.73747913179328}

In [8]:
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.metrics import mean_absolute_error as MAE

In [9]:
# 전체 특징을 모두 사용하였을 때
model = MLP(random_state = 2313, max_iter = 500)
model.fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)
score = MAE(Test_Y, pred_Y)
print(score)

1.5276345561460773


In [10]:
# VIF 점수가 30점 미만인 특징만 사용하였을 때 
selected_features = [key for key, val in VIF_dict.items() if val < 30] 
model = MLP(random_state = 2313, max_iter = 500)
model.fit(Train_X[selected_features], Train_Y)
pred_Y = model.predict(Test_X[selected_features])
score = MAE(Test_Y, pred_Y)
print(score)

1.5076713980631804


#### PCA 사용

In [12]:
from sklearn.decomposition import PCA
PCA_model = PCA(n_components = 3).fit(Train_X)

Train_Z = PCA_model.transform(Train_X)
Test_Z = PCA_model.transform(Test_X)

print(Train_Z.shape)

(3132, 3)


In [13]:
model = MLP(random_state = 2313, max_iter = 500)
model.fit(Train_Z, Train_Y)
pred_Y = model.predict(Test_Z)
score = MAE(Test_Y, pred_Y)
print(score)

1.4484133888721893
